# Vector Search 2.0 Experiment: Slide Library

This notebook tests Vector Search 2.0 for managing presentation slide libraries.

**Run this in Google Colab for easiest setup!**

## Setup

In [ ]:
# Install required packages
!pip install google-cloud-aiplatform pdf2image Pillow --quiet

# Authenticate (in Colab, this will prompt for login)
from google.colab import auth
auth.authenticate_user()

# Set your project ID
PROJECT_ID = 'deckr-477706'
LOCATION = 'us-central1'

!gcloud config set project {PROJECT_ID}

## Step 1: Generate Embeddings for Slides

In [ ]:
from vertexai.vision_models import Image, MultiModalEmbeddingModel
import vertexai
import base64
from io import BytesIO
from PIL import Image as PILImage

# Initialize Vertex AI
vertexai.init(project=PROJECT_ID, location=LOCATION)

# Load embedding model
model = MultiModalEmbeddingModel.from_pretrained("multimodalembedding@001")

def generate_slide_embedding(image_path: str, description: str = "presentation slide"):
    """Generate embedding for a slide image"""
    image = Image.load_from_file(image_path)
    embeddings = model.get_embeddings(
        image=image,
        contextual_text=description,
        dimension=512  # Use 512 for faster processing
    )
    return embeddings.image_embedding

print("✅ Embedding function ready")

## Step 2: Upload Your PDF

Upload your `title_slide.pdf` using the file upload button in Colab.

In [ ]:
from google.colab import files
from pdf2image import convert_from_path
import tempfile
import os

# Upload PDF
print("📤 Upload your PDF file:")
uploaded = files.upload()

# Get the PDF filename
pdf_filename = list(uploaded.keys())[0]
print(f"\n✅ Uploaded: {pdf_filename}")

# Convert PDF to images
print("\n🔄 Converting PDF to images...")
images = convert_from_path(pdf_filename)

# Save images temporarily
temp_dir = tempfile.mkdtemp()
image_paths = []

for i, image in enumerate(images):
    image_path = os.path.join(temp_dir, f"slide_{i+1}.png")
    image.save(image_path, 'PNG')
    image_paths.append(image_path)

print(f"✅ Extracted {len(images)} slides")

# Display first slide
print("\n📸 Preview of Slide 1:")
images[0]

## Step 3: Generate Embeddings for All Slides

In [ ]:
import time

# Generate embeddings for each slide
slide_data = []

print("📊 Generating embeddings for all slides...\n")

for i, image_path in enumerate(image_paths):
    print(f"Processing slide {i+1}/{len(image_paths)}...", end=" ")
    start_time = time.time()
    
    # Generate embedding
    embedding = generate_slide_embedding(image_path, "presentation slide")
    
    # Store data
    slide_data.append({
        'id': f'slide-{i+1}',
        'name': f'Slide {i+1}',
        'slide_number': i+1,
        'image_path': image_path,
        'embedding': embedding,
        'deck_name': pdf_filename
    })
    
    elapsed = time.time() - start_time
    print(f"✅ ({elapsed:.2f}s)")

print(f"\n✅ Generated embeddings for {len(slide_data)} slides")
print(f"📏 Embedding dimension: {len(slide_data[0]['embedding'])}")

## Step 4: Query for "Title Slide"

In [ ]:
import numpy as np

def cosine_similarity(a, b):
    """Calculate cosine similarity between two vectors"""
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def query_slides(query_text: str, top_k: int = 5):
    """Query slides by text"""
    print(f"🔍 Query: \"{query_text}\"\n")
    
    # Generate embedding for query text
    query_embeddings = model.get_embeddings(
        contextual_text=query_text,
        dimension=512
    )
    query_embedding = query_embeddings.text_embedding
    
    # Calculate similarity with all slides
    results = []
    for slide in slide_data:
        similarity = cosine_similarity(query_embedding, slide['embedding'])
        results.append({
            **slide,
            'similarity': similarity
        })
    
    # Sort by similarity
    results.sort(key=lambda x: x['similarity'], reverse=True)
    
    # Return top K
    return results[:top_k]

# Test query
results = query_slides("title slide cover page presentation")

# Display results
print("📊 Top Results:\n")
for i, result in enumerate(results):
    print(f"[{i+1}] Slide {result['slide_number']}")
    print(f"    Similarity: {result['similarity']:.4f}")
    print(f"    Name: {result['name']}")
    print()

## Step 5: Visualize Top Results

In [ ]:
import matplotlib.pyplot as plt

# Show top 3 results
fig, axes = plt.subplots(1, min(3, len(results)), figsize=(15, 5))
if len(results) == 1:
    axes = [axes]

for i, result in enumerate(results[:3]):
    img = PILImage.open(result['image_path'])
    axes[i].imshow(img)
    axes[i].set_title(f"Slide {result['slide_number']}\nSimilarity: {result['similarity']:.4f}")
    axes[i].axis('off')

plt.tight_layout()
plt.show()

## Step 6: Test Different Queries

In [ ]:
# Test different query types
test_queries = [
    "title slide",
    "content slide with bullet points",
    "data visualization chart graph",
    "section divider",
    "closing slide thank you"
]

print("🧪 Testing Multiple Queries\n")
print("=" * 60)

for query in test_queries:
    results = query_slides(query, top_k=3)
    print(f"\nQuery: \"{query}\"")
    print("Top matches:")
    for i, r in enumerate(results):
        print(f"  {i+1}. Slide {r['slide_number']} (similarity: {r['similarity']:.4f})")
    print()

## 📊 Experiment Results

### What to Look For:

1. **Precision**: Does "title slide" query return actual title slides?
2. **Recall**: Are all title slides ranked high?
3. **False Positives**: Do section dividers/closing slides rank too high?
4. **Performance**: How long did embedding generation take?

### Expected Issues:

- Visual similarity might cause confusion (title slides look like section dividers)
- Need explicit categories for reliable filtering

### Next Steps:

If results are promising → Implement Vector Search 2.0 with category labels
If results are poor → Stick with explicit categorization approach